paper: https://link.springer.com/article/10.1007/s11111-024-00452-9#Sec10

Prophet: https://facebook.github.io/prophet/docs/quick_start.html#python-api  
github: https://github.com/facebook/prophet  

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path

# Geospatial
import geopandas as gpd
import dask_geopandas as dgpd
import rasterio
from rasterstats import zonal_stats
from osgeo import gdal, osr

# Statistics & Bayesian modeling
import pymc as pm
import arviz as az

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Base directories
BASE_DIR = Path("/Users/wenlanzhang/PycharmProjects/Mapineq/src/")
DATA_DIR = Path("/Users/wenlanzhang/Downloads/PhD_UCL/Data/Oxford")

In [ ]:
df = pd.read_csv(DATA_DIR/f"Migration/international_migration_flow.csv") 
df['year'] = pd.to_datetime(df['migration_month']).dt.year
df['month'] = pd.to_datetime(df['migration_month']).dt.month
df["migration_month"] = pd.to_datetime(df["migration_month"])

# Define the mapping of old codes to new codes
country_code_mapping = {
    'GR': 'EL',  # Greece (GR → EL)
    'GB': 'UK'   # United Kingdom (GB → UK)
}

# Apply the replacement to both columns
df['country_from'] = df['country_from'].replace(country_code_mapping)
df['country_to'] = df['country_to'].replace(country_code_mapping)

df
# len(df['country_from'].unique())

In [ ]:
# country_list = ['EL', 'ES', 'FI', 'FR', 'HR', 'EE', 'DE', 'DK', 'BA', 'AT', 'BG',
#        'CH', 'CY', 'BE', 'CZ', 'AL', 'LU', 'LV', 'ME', 'IE', 'IS', 'IT',
#        'MK', 'MT', 'LI', 'NL', 'LT', 'HU', 'RS', 'SE', 'SI', 'TR', 'UA',
#        'SK', 'RO', 'NO', 'PL', 'PT', 'XK', 'UK']
# # df = df[(df['country_from'].isin(country_list)) | (df['country_to'].isin(country_list))]
# # df = df[(df['country_from'].isin(country_list)) & (df['country_to'].isin(country_list))]

# # df = df[(df['country_from'] == 'UA') & (df['country_to'].isin(country_list))]
# # df = df[df['num_migrants'] != 0]
# df = df[df['num_migrants'] >= 100]

# df

# Plot

## UA 

In [ ]:
# 1. Filter only rows where destination is UA
df_ua_in = df[df['country_to'] == 'UA']
df_ua_out = df[df['country_from'] == 'UA']

In [ ]:
# # Plot one line per origin country
# plt.figure(figsize=(8, 5))
# for country in df_ua_in['country_from'].unique():
#     subset = df_ua_in[df_ua_in['country_from'] == country]
#     plt.plot(subset['migration_month'], subset['num_migrants'], label=country)
#     # plt.plot(subset['migration_month'], subset['num_migrants'], marker='o', label=country)

# # 3. Add labels and style
# plt.title("Migrants to UA Over Time")
# plt.xlabel("Month")
# plt.ylabel("Number of Migrants")
# # plt.legend(title="From Country")
# plt.grid(True)
# plt.tight_layout()

# plt.show()

In [ ]:
# df_ua_in
fig = px.line(
    df_ua_in,
    x="migration_month",
    y="num_migrants",
    color="country_from",  # separate line for each origin country
    markers=True,
    title="Migrants to UA Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_ua_out
fig = px.line(
    df_ua_out,
    x="migration_month",
    y="num_migrants",
    color="country_to",  # separate line for each origin country
    markers=True,
    title="Migrants from UA Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### From PL  

In [ ]:
# df_pl_out
df_pl_out = df[(df['country_from'] == 'PL') & (df['num_migrants'] != 0)]

fig = px.line(
    df_pl_out,
    x="migration_month",
    y="num_migrants",
    color="country_to",  # separate line for each origin country
    markers=True,
    title="Migrants from PL Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### From DE  

In [ ]:
# df_de_out
df_de_out = df[(df['country_from'] == 'DE') & (df['num_migrants'] != 0)]

fig = px.line(
    df_de_out,
    x="migration_month",
    y="num_migrants",
    color="country_to",  # separate line for each origin country
    markers=True,
    title="Migrants from DE Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### From CZ  

In [ ]:
# df_cz_out
df_cz_out = df[(df['country_from'] == 'CZ') & (df['num_migrants'] != 0)]

fig = px.line(
    df_cz_out,
    x="migration_month",
    y="num_migrants",
    color="country_to",  # separate line for each origin country
    markers=True,
    title="Migrants from CZ Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
df_SY_in = df[(df['country_to'] == 'SY') & (df['num_migrants'] != 0)]
df_SY_out = df[(df['country_from'] == 'SY') & (df['num_migrants'] != 0)]

# df_DO_in
fig = px.line(
    df_SY_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to SY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_SY_out
fig = px.line(
    df_SY_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from SY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
df_TR_in = df[(df['country_to'] == 'TR') & (df['num_migrants'] != 0)]
df_TR_out = df[(df['country_from'] == 'TR') & (df['num_migrants'] != 0)]

# df_TR_in
fig = px.line(
    df_TR_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to TR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_TR_out
fig = px.line(
    df_TR_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from TR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()